# Jumper Media Analytics - Main Analysis Dashboard

This notebook contains comprehensive analysis of engagement data including:
- Top authors and categories by engagement
- Time-of-day and day-of-week patterns
- Opportunity identification (high volume, low engagement)
- Visualizations and insights


## 1. Setup and Imports


In [ ]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Import config
from config import DB_CONFIG


In [ ]:
# Database connection helper
def get_db_connection():
    return psycopg2.connect(**DB_CONFIG)

def execute_query(query, connection=None):
    """Execute SQL query and return DataFrame"""
    if connection is None:
        conn = get_db_connection()
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    else:
        return pd.read_sql_query(query, connection)


## 2. Executive Summary


In [ ]:
# Overall metrics
conn = get_db_connection()

summary_query = """
SELECT 
    COUNT(DISTINCT a.author_id) AS total_authors,
    COUNT(DISTINCT p.post_id) AS total_posts,
    COUNT(e.engagement_id) AS total_engagements,
    COUNT(*) FILTER (WHERE e.type = 'view') AS total_views,
    COUNT(*) FILTER (WHERE e.type = 'like') AS total_likes,
    COUNT(*) FILTER (WHERE e.type = 'comment') AS total_comments,
    COUNT(*) FILTER (WHERE e.type = 'share') AS total_shares,
    ROUND(COUNT(e.engagement_id)::NUMERIC / NULLIF(COUNT(DISTINCT p.post_id), 0), 2) AS avg_engagement_per_post
FROM authors a
LEFT JOIN posts p ON a.author_id = p.author_id
LEFT JOIN engagements e ON p.post_id = e.post_id
WHERE p.publish_timestamp >= CURRENT_DATE - INTERVAL '30 days'
"""

summary = execute_query(summary_query, conn)
print("=== Last 30 Days Summary ===")
print(summary.to_string(index=False))

conn.close()


## 3. Top Authors and Categories by Engagement


In [ ]:
# Query 1: Top Authors (Last 30 Days)
conn = get_db_connection()

top_authors_query = """
SELECT 
    a.author_id,
    a.name,
    a.author_category,
    COUNT(*) FILTER (WHERE e.type = 'view') AS total_views,
    COUNT(*) FILTER (WHERE e.type = 'like') AS total_likes,
    COUNT(*) FILTER (WHERE e.type = 'comment') AS total_comments,
    COUNT(*) FILTER (WHERE e.type = 'share') AS total_shares,
    COUNT(*) AS total_engagements,
    COUNT(DISTINCT e.post_id) AS posts_with_engagement,
    ROUND(COUNT(*)::NUMERIC / NULLIF(COUNT(DISTINCT e.post_id), 0), 2) AS avg_engagement_per_post
FROM authors a
JOIN posts p ON a.author_id = p.author_id
JOIN engagements e ON p.post_id = e.post_id
WHERE e.engaged_timestamp >= CURRENT_DATE - INTERVAL '30 days'
GROUP BY a.author_id, a.name, a.author_category
ORDER BY total_engagements DESC
LIMIT 20
"""

top_authors = execute_query(top_authors_query, conn)
print("Top 20 Authors by Engagement (Last 30 Days)")
print(top_authors.head(10).to_string(index=False))


In [ ]:
# Visualization: Top Authors Bar Chart
fig, ax = plt.subplots(figsize=(14, 8))

top_10 = top_authors.head(10)
x = range(len(top_10))
width = 0.6

ax.bar(x, top_10['total_engagements'], width, label='Total Engagements', color='steelblue')
ax.set_xlabel('Author', fontsize=12)
ax.set_ylabel('Total Engagements', fontsize=12)
ax.set_title('Top 10 Authors by Total Engagement (Last 30 Days)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(top_10['name'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/top_authors.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Top Categories
top_categories_query = """
SELECT 
    p.category,
    COUNT(*) FILTER (WHERE e.type = 'view') AS total_views,
    COUNT(*) FILTER (WHERE e.type = 'like') AS total_likes,
    COUNT(*) FILTER (WHERE e.type = 'comment') AS total_comments,
    COUNT(*) FILTER (WHERE e.type = 'share') AS total_shares,
    COUNT(*) AS total_engagements,
    COUNT(DISTINCT e.post_id) AS posts_with_engagement,
    COUNT(DISTINCT p.author_id) AS unique_authors,
    ROUND(COUNT(*)::NUMERIC / NULLIF(COUNT(DISTINCT e.post_id), 0), 2) AS avg_engagement_per_post,
    ROUND((COUNT(*)::NUMERIC / SUM(COUNT(*)) OVER ()) * 100, 2) AS engagement_percentage
FROM posts p
JOIN engagements e ON p.post_id = e.post_id
WHERE e.engaged_timestamp >= CURRENT_DATE - INTERVAL '30 days'
GROUP BY p.category
ORDER BY total_engagements DESC
"""

top_categories = execute_query(top_categories_query, conn)
print("\nCategory Performance (Last 30 Days)")
print(top_categories.to_string(index=False))


In [ ]:
# Visualization: Category Performance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Total engagements by category
ax1.barh(top_categories['category'], top_categories['total_engagements'], color='coral')
ax1.set_xlabel('Total Engagements', fontsize=12)
ax1.set_title('Total Engagements by Category', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Average engagement per post
ax2.barh(top_categories['category'], top_categories['avg_engagement_per_post'], color='lightgreen')
ax2.set_xlabel('Avg Engagement per Post', fontsize=12)
ax2.set_title('Average Engagement per Post by Category', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/category_performance.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Engagement Trends Over Time
trends_query = """
SELECT 
    DATE_TRUNC('month', e.engaged_timestamp) AS month,
    a.author_id,
    a.name,
    COUNT(*) AS total_engagements
FROM authors a
JOIN posts p ON a.author_id = p.author_id
JOIN engagements e ON p.post_id = e.post_id
WHERE e.engaged_timestamp >= CURRENT_DATE - INTERVAL '6 months'
GROUP BY DATE_TRUNC('month', e.engaged_timestamp), a.author_id, a.name
ORDER BY month DESC, total_engagements DESC
"""

trends = execute_query(trends_query, conn)
trends['month'] = pd.to_datetime(trends['month'])

# Get top 5 authors for trend visualization
top_author_ids = top_authors.head(5)['author_id'].tolist()
trends_top = trends[trends['author_id'].isin(top_author_ids)]

# Pivot for easier plotting
trends_pivot = trends_top.pivot(index='month', columns='name', values='total_engagements').fillna(0)


In [ ]:
# Visualization: Engagement Trends
fig, ax = plt.subplots(figsize=(14, 8))

for author in trends_pivot.columns:
    ax.plot(trends_pivot.index, trends_pivot[author], marker='o', label=author, linewidth=2)

ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Total Engagements', fontsize=12)
ax.set_title('Engagement Trends: Top 5 Authors (Last 6 Months)', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(alpha=0.3)
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../visualizations/engagement_trends.png', dpi=300, bbox_inches='tight')
plt.show()

conn.close()


## 4. Time-of-Day and Day-of-Week Patterns


In [ ]:
# Query 2: Time Patterns
conn = get_db_connection()

# Engagement by hour
hour_query = """
SELECT 
    EXTRACT(HOUR FROM engaged_timestamp) AS hour_of_day,
    COUNT(*) AS total_engagements,
    COUNT(*) FILTER (WHERE type = 'view') AS views,
    COUNT(*) FILTER (WHERE type = 'like') AS likes,
    COUNT(*) FILTER (WHERE type = 'comment') AS comments,
    COUNT(*) FILTER (WHERE type = 'share') AS shares
FROM engagements
GROUP BY EXTRACT(HOUR FROM engaged_timestamp)
ORDER BY hour_of_day
"""

hour_data = execute_query(hour_query, conn)
print("Engagement by Hour of Day")
print(hour_data.to_string(index=False))


In [ ]:
# Visualization: Engagement by Hour
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(hour_data['hour_of_day'], hour_data['total_engagements'], 
        marker='o', linewidth=2, markersize=8, color='steelblue')
ax.fill_between(hour_data['hour_of_day'], hour_data['total_engagements'], 
                alpha=0.3, color='steelblue')

ax.set_xlabel('Hour of Day', fontsize=12)
ax.set_ylabel('Total Engagements', fontsize=12)
ax.set_title('Engagement Patterns by Hour of Day', fontsize=14, fontweight='bold')
ax.set_xticks(range(0, 24))
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/engagement_by_hour.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Engagement by day of week
dow_query = """
SELECT 
    EXTRACT(DOW FROM engaged_timestamp) AS day_number,
    TO_CHAR(engaged_timestamp, 'Day') AS day_of_week,
    COUNT(*) AS total_engagements
FROM engagements
GROUP BY EXTRACT(DOW FROM engaged_timestamp), TO_CHAR(engaged_timestamp, 'Day')
ORDER BY day_number
"""

dow_data = execute_query(dow_query, conn)
dow_data['day_of_week'] = dow_data['day_of_week'].str.strip()
print("\nEngagement by Day of Week")
print(dow_data.to_string(index=False))


In [ ]:
# Visualization: Engagement by Day of Week
fig, ax = plt.subplots(figsize=(12, 6))

ax.bar(dow_data['day_of_week'], dow_data['total_engagements'], color='teal')
ax.set_xlabel('Day of Week', fontsize=12)
ax.set_ylabel('Total Engagements', fontsize=12)
ax.set_title('Engagement Patterns by Day of Week', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../visualizations/engagement_by_day.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Heatmap: Engagement by Hour and Day of Week
heatmap_query = """
SELECT 
    EXTRACT(DOW FROM engaged_timestamp) AS day_of_week,
    EXTRACT(HOUR FROM engaged_timestamp) AS hour_of_day,
    COUNT(*) AS engagement_count
FROM engagements
GROUP BY EXTRACT(DOW FROM engaged_timestamp), EXTRACT(HOUR FROM engaged_timestamp)
ORDER BY day_of_week, hour_of_day
"""

heatmap_data = execute_query(heatmap_query, conn)

# Create pivot table for heatmap
heatmap_pivot = heatmap_data.pivot(index='day_of_week', columns='hour_of_day', values='engagement_count').fillna(0)

# Map day numbers to names
day_names = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
heatmap_pivot.index = [day_names[int(i)] for i in heatmap_pivot.index]


In [ ]:
# Visualization: Heatmap
fig, ax = plt.subplots(figsize=(16, 8))

sns.heatmap(heatmap_pivot, annot=True, fmt='.0f', cmap='YlOrRd', 
            cbar_kws={'label': 'Engagement Count'}, ax=ax)

ax.set_xlabel('Hour of Day', fontsize=12)
ax.set_ylabel('Day of Week', fontsize=12)
ax.set_title('Engagement Heatmap: Day of Week vs Hour of Day', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../visualizations/engagement_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

conn.close()


## 5. Opportunity Analysis: High Volume, Low Engagement


In [ ]:
# Query 3: Opportunity Areas
conn = get_db_connection()

opportunity_query = """
WITH author_stats AS (
    SELECT 
        a.author_id,
        a.name,
        a.author_category,
        COUNT(DISTINCT p.post_id) AS total_posts,
        COUNT(e.engagement_id) AS total_engagements,
        ROUND(COUNT(e.engagement_id)::NUMERIC / NULLIF(COUNT(DISTINCT p.post_id), 0), 2) AS avg_engagement_per_post
    FROM authors a
    LEFT JOIN posts p ON a.author_id = p.author_id
    LEFT JOIN engagements e ON p.post_id = e.post_id
    WHERE p.publish_timestamp >= CURRENT_DATE - INTERVAL '90 days'
    GROUP BY a.author_id, a.name, a.author_category
),
overall_avg AS (
    SELECT AVG(avg_engagement_per_post) AS overall_avg_engagement
    FROM author_stats
    WHERE total_posts > 0
)
SELECT 
    as.author_id,
    as.name,
    as.author_category,
    as.total_posts,
    as.total_engagements,
    as.avg_engagement_per_post,
    oa.overall_avg_engagement,
    ROUND(((as.avg_engagement_per_post - oa.overall_avg_engagement) / NULLIF(oa.overall_avg_engagement, 0)) * 100, 2) AS performance_vs_avg_percent
FROM author_stats as
CROSS JOIN overall_avg oa
WHERE as.total_posts > 0
ORDER BY as.total_posts DESC, as.avg_engagement_per_post ASC
"""

opportunity_data = execute_query(opportunity_query, conn)
print("Author Performance: Volume vs Engagement")
print(opportunity_data.head(20).to_string(index=False))


In [ ]:
# Scatter Plot: Volume vs Engagement
fig, ax = plt.subplots(figsize=(14, 8))

# Color code by performance
colors = []
for idx, row in opportunity_data.iterrows():
    if row['avg_engagement_per_post'] < row['overall_avg_engagement']:
        colors.append('red')  # Underperforming
    else:
        colors.append('green')  # Performing well

scatter = ax.scatter(opportunity_data['total_posts'], 
                    opportunity_data['avg_engagement_per_post'],
                    c=colors, alpha=0.6, s=100, edgecolors='black', linewidth=1)

# Add average line
if len(opportunity_data) > 0:
    avg_engagement = opportunity_data['overall_avg_engagement'].iloc[0]
    ax.axhline(y=avg_engagement, color='blue', linestyle='--', 
               label=f'Overall Average ({avg_engagement:.2f})', linewidth=2)

ax.set_xlabel('Post Volume (Total Posts)', fontsize=12)
ax.set_ylabel('Average Engagement per Post', fontsize=12)
ax.set_title('Opportunity Analysis: Post Volume vs Engagement Rate', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Annotate top underperformers
underperformers = opportunity_data[opportunity_data['avg_engagement_per_post'] < opportunity_data['overall_avg_engagement']]
top_underperformers = underperformers.nlargest(5, 'total_posts')

for idx, row in top_underperformers.iterrows():
    ax.annotate(row['name'], 
                (row['total_posts'], row['avg_engagement_per_post']),
                xytext=(5, 5), textcoords='offset points', fontsize=9,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.savefig('../visualizations/opportunity_scatter.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Identify top opportunity authors
top_opportunities = opportunity_data[
    (opportunity_data['total_posts'] >= opportunity_data['total_posts'].quantile(0.75)) &
    (opportunity_data['avg_engagement_per_post'] < opportunity_data['overall_avg_engagement'])
].sort_values('total_posts', ascending=False)

print("\n=== Top Opportunity Authors (High Volume, Low Engagement) ===")
print(top_opportunities[['name', 'author_category', 'total_posts', 
                         'avg_engagement_per_post', 'performance_vs_avg_percent']].to_string(index=False))

conn.close()


## 6. Key Insights Summary


In [ ]:
print("=== KEY INSIGHTS ===\n")

print("1. TOP PERFORMERS:")
if len(top_authors) > 0:
    print(f"   - Top author: {top_authors.iloc[0]['name']} with {top_authors.iloc[0]['total_engagements']} engagements")
if len(top_categories) > 0:
    print(f"   - Top category: {top_categories.iloc[0]['category']} with {top_categories.iloc[0]['total_engagements']} engagements\n")

print("2. TEMPORAL PATTERNS:")
if len(hour_data) > 0:
    peak_hour = hour_data.loc[hour_data['total_engagements'].idxmax(), 'hour_of_day']
    print(f"   - Peak engagement hour: {int(peak_hour)}:00")
if len(dow_data) > 0:
    peak_day = dow_data.loc[dow_data['total_engagements'].idxmax(), 'day_of_week']
    print(f"   - Peak engagement day: {peak_day}\n")

print("3. OPPORTUNITIES:")
if len(top_opportunities) > 0:
    print(f"   - Found {len(top_opportunities)} high-volume authors with below-average engagement")
    print(f"   - Top opportunity: {top_opportunities.iloc[0]['name']} ({top_opportunities.iloc[0]['total_posts']} posts, {top_opportunities.iloc[0]['avg_engagement_per_post']:.2f} avg engagement)")
else:
    print("   - No significant opportunities identified with current data")

print("\n=== See recommendations.md for detailed recommendations ===")
